# Clasificación con Support Vector Machines

SVM es un algoritmo de Machine Learning que se utiliza en problemas de clasificación. Los algoritmos SVM se encargan de encontrar un separador, es decir, una curva, plano o hiperplano que permita separar las categorías. 

Los algoritmos de SVM mapean los datos a un espacio de más dimensiones que permitan visualizar las categorías con más facilidad. 

Por ejemplo, supongamos que tenemos dos categorías y que al representar dos de sus características una frente a la otra vemos que hay una separación entre las categorías pero que no es lineal, sino que se dibuja una curva como vemos a continuación:

<img src="nolinear.png">

En lugar de trabajar con dos dimensiones, vamos a construir un espacio tridimensional:

<img src="hiperplano.png">

Como vemos en esta imagen, ahora las categorías están separadas completamente por un plano bidimensional. 

En general, un algoritmos SVM buscará un hiperplano óptimo que permita la separación de las categorías de forma precisa. El hecho de mapear los datos a dimensiones superiores se denomina <b>kernelling</b>. 

Tenemos dos principales ventajas a la hora de trabajar con SVM:  Funciona bien en espacios de varias dimensiones y además es eficiente a nivel computacional.

# Clasificando muestras en benignas o malignas usando SVM

In [53]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
%matplotlib inline 
import matplotlib.pyplot as plt

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

%precision 4

'%.4f'

In [54]:
df = pd.read_csv('muestras_celulas.csv')
df.head()

,ID,Clump,UnifSize,UnifShape,MargAdh,SingEpiSize,BareNuc,BlandChrom,NormNucl,Mit,Class
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2


A partir de las variables que tenemos, debemos predecir la columna "Class". Los valores que toma son "benigno" (2) o "maligno" (4).

In [55]:
print(df.isnull().sum())
print("--------")
print(df.dtypes)

ID             0
Clump          0
UnifSize       0
UnifShape      0
MargAdh        0
SingEpiSize    0
BareNuc        0
BlandChrom     0
NormNucl       0
Mit            0
Class          0
dtype: int64
--------
ID              int64
Clump           int64
UnifSize        int64
UnifShape       int64
MargAdh         int64
SingEpiSize     int64
BareNuc        object
BlandChrom      int64
NormNucl        int64
Mit             int64
Class           int64
dtype: object


In [56]:
df = df[pd.to_numeric(df['BareNuc'], errors='coerce').notnull()]
df['BareNuc']=df['BareNuc'].astype('int')

In [57]:
#Vamos a elegir nuestras variables de entrada y la variable de salida
variables= ['Clump','UnifSize','UnifShape','MargAdh','SingEpiSize','BareNuc','BlandChrom','NormNucl','Mit']
X=df[variables]
Y=df['Class']

In [93]:
#Vamos a separar el set de datos en entrenamiento y testeo
X_train, X_test, Y_train, Y_test = train_test_split( X, Y, test_size=0.2, random_state=4, stratify=Y)
print ('Set de entrenamiento (dimensiones) :', X_train.shape,  Y_train.shape)
print ('Set de testeo (dimensiones): ', X_test.shape,  Y_test.shape)

Set de entrenamiento (dimensiones) : (546, 9) (546,)
Set de testeo (dimensiones):  (137, 9) (137,)


In [94]:
#Vamos a construir el modelo SVM usando sklearn con una función de base radial (kernel=rbf)
from sklearn import svm
model = svm.SVC(kernel='rbf')
model.fit(X_train,Y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [95]:
#Predicción del modelo
Y_pred = model.predict(X_test)
print(Y_pred)

[2 2 4 2 4 2 2 2 2 2 2 4 4 2 4 4 4 2 4 4 2 2 2 2 2 2 2 4 2 4 2 2 4 2 2 4 4
 4 2 2 2 4 2 2 2 4 2 2 2 2 2 4 2 4 2 4 4 4 4 4 4 2 2 2 2 2 2 2 2 2 4 2 2 2
 2 2 4 2 2 2 2 2 4 4 2 4 2 2 4 2 4 2 4 2 2 4 4 4 2 2 4 2 4 4 2 2 4 4 4 2 4
 2 2 2 2 4 2 2 4 2 2 4 4 2 4 4 2 4 2 2 4 4 2 2 4 2 2]


In [96]:
#Evaluación del modelo usando F1-Score y el índice Jaccard
from sklearn.metrics import f1_score, jaccard_score
F1= f1_score(Y_test, Y_pred, average='weighted')
Jac = jaccard_score(Y_test, Y_pred,average='weighted')
print('El F1-Score es: F1-Score= '+str(F1)+' y el índice Jaccard es: J='+str(Jac))

El F1-Score es: F1-Score= 0.9638796207370622 y el índice Jaccard es: J=0.930450351191296


In [97]:
from sklearn.metrics import classification_report, confusion_matrix
matriz = confusion_matrix(Y_test, Y_pred, labels=[2,4])
print('La matriz de confusión es \n ' + str(matriz))
print('-------')
print(classification_report(Y_test,Y_pred))

La matriz de confusión es 
 [[84  5]
 [ 0 48]]
-------
              precision    recall  f1-score   support

           2       1.00      0.94      0.97        89
           4       0.91      1.00      0.95        48

    accuracy                           0.96       137
   macro avg       0.95      0.97      0.96       137
weighted avg       0.97      0.96      0.96       137



# Una vez finalizado... 

Podemos hacer diferentes pruebas y en lugar de probar con los diferentes kernels que hay:

In [111]:
kernels = ['linear','poly','rbf','sigmoid']
f1_total = []
jaccard_total = []
for i in kernels:
    modelo =svm.SVC(kernel=str(i))
    modelo.fit(X_train,Y_train)
    Y_kernel = modelo.predict(X_test)
    F1_bucle= f1_score(Y_test, Y_kernel, average='weighted')
    Jac_bucle = jaccard_score(Y_test, Y_kernel, average='weighted')
    f1_total.append(F1_bucle)
    jaccard_total.append(Jac_bucle)
print(f1_total)
print(jaccard_total)

[0.9492413182924131, 0.904875122602412, 0.9638796207370622, 0.3371079108305386]
[0.9037534562944286, 0.8276814679643147, 0.930450351191296, 0.22760935585273592]


In [118]:
kernels2 = ['Lineal','Polinómico','RBF','Sigmoide']

scores = pd.DataFrame(list(zip(kernels2,f1_total, jaccard_total)), 
               columns =['Kernel', 'F1','Jaccard']) 

In [119]:
scores

,Kernel,F1,Jaccard
0,Lineal,0.949241,0.903753
1,Polinómico,0.904875,0.827681
2,RBF,0.963880,0.930450
3,Sigmoide,0.337108,0.227609
